### Notebook to illustrate MCMC sampling of Norms from pre-defined PCFGs

 *There are  the 5 main sections :*  
 **1. Initialising Environment and True Expression**  
 **2. Create Action Data on environment while performing randomised tasks**  
 **3. Run MCMC Algorithms to learn expressions from the data created earlier**  
 **4. Test  the performance of MCMC Algorithm by calculating Preciscion and Recall of Learned Norms**  
 **5. Test the convergence of MCMC algorithm**  

In [ ]:
#Import the different modules required
from mcmc_norm_learning.environment import *
from mcmc_norm_learning.rules_4 import *
from mcmc_norm_learning.robot_task_new import *
from mcmc_norm_learning.algorithm_1_v4 import create_data,algorithm_1,to_tuple
from mcmc_norm_learning.mcmc_performance import performance
from mcmc_norm_learning.mcmc_convergence import prepare_sequences,calculate_R

import matplotlib.pyplot as plt
from collections import Counter
import pickle
import time
import seaborn as sns
import os
import sys
from tqdm import tnrange, tqdm_notebook


### 1. Initialising Environment and True Expression
* *Environment can be initialisaed with any number of objects (default=20) and a seed value can also be fed.*
* *Expression refers to Norms initialised from the PCFG, True Expression means the eexpression used to create data which if further used to learn norms from MCMC.*


In [ ]:
actionable = []
while len(actionable) < 4:
    env=create_env(N=40)
    #fig,ax=plt.subplots(figsize=(8,6))
    #plot_env(env,ax,legend=True)

    #Dump env to file
    with open('./demo/demo_env.sv', 'wb') as fp:
        pickle.dump(env, fp)

    target_area=[position(-0.8,0.7),position(0.25,0.99)]
    task1=task(colour_specific=np.nan,shape_specific=np.nan,target_area=target_area)

    rob = robot(task1,env)
    actionable = rob.all_actionable()

print(actionable)

fig,ax=plt.subplots(figsize=(8,6))
plot_task(env,ax,"Clearing the highlighted area",task1,True)
#fig.savefig("example_task.pdf", bbox_inches='tight')



In [ ]:
true_expression=expand("NORMS")
print_expression(true_expression)


#Dump expression
with open('./demo/demo_exp.sv', 'wb') as fp:
    pickle.dump(true_expression, fp)

### 2. Create Action Data on environment while performing randomised tasks
Action data is can be created in two ways (parametrised by random_task):
1. Either Initialising a task beforehand and performing it ceratin times (*num_repeat*)
2. or, by initialisng random tasks for each iteration in *num_repeat*. In such a case relevance of tasks becomes redundant, and though target are of task is randomised, scope of task (i.e. color and shape of objects) is fixed. And in next step rf must be passed as nan.

In both the cases the repetition of tasks is on the original state of environment provided to function

In [ ]:
true_expression = ['Rules', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '2'], ['Moved', ['Colour', 'g'], ['Shape', 'square'], ['Zone', '2'], ['Next-Move', ['Colour', 'b'], ['Shape', 'triangle']]]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'g'], ['Shape', 'triangle'], ['PerZone', '1']]]

with open('./demo/demo_exp.sv', 'wb') as fp:
    pickle.dump(true_expression, fp)

In [ ]:
"""
with open('./demo/demo_env.sv', 'wb') as fp:
    env = pickle.load(fp)
with open('./demo/demo_exp.sv', 'rb') as fp:
    true_expression = pickle.load(fp)
"""

import mcmc_norm_learning.verify_action_4
import mcmc_norm_learning.rules_4
import mcmc_norm_learning.robot_task_new
import mcmc_norm_learning.working
from importlib import reload
from copy import deepcopy

reload(mcmc_norm_learning.verify_action_4)
reload(mcmc_norm_learning.rules_4)
reload(mcmc_norm_learning.robot_task_new)
reload(mcmc_norm_learning.working)
from mcmc_norm_learning.working import *
from mcmc_norm_learning.robot_task_new import *
from mcmc_norm_learning.verify_action_4 import *

rules = [true_expression[i] for i in range(1, len(true_expression))]

if rules[0][0]=="Pro":
    the_pro = rules[0]
    print(">>> Prohibition match check for", the_pro, '\n')
    for oid in actionable:
        get_obj(oid,env).describe()
        print(check_pro_or_per(get_obj(oid,env),pro_or_per_action(the_pro),{1:the_pro}))
        print()

if rules[0][0]=="Obl":
    rule = rules[0]
    print(">>> Obligation match check for", rule, '\n')
    cond = rule[1]
    conds_list = separate_conds(cond)
    next_move = conds_list[-1]
    for oid in actionable:
        get_obj(oid,env).describe()
        print(check_obl(get_obj(oid,env),{1:rule}))
        print()

if len(rules) > 1:
    the_per = rules[1]
    print(">>> Permission match check for", the_per,'\n')
    for oid in actionable:
        get_obj(oid,env).describe()
        print(check_pro_or_per(get_obj(oid,env),pro_or_per_action(the_per),{1:the_per}))
        print()

rob = robot(task1,deepcopy(env))
ac = rob.all_compliant(rules,"foo")
print(ac)

from working import unless_moves
print("List of unless moves")
print(list(unless_moves(ac)))

order_constrained_subseqs = []
for um in unless_moves(ac):
    obl_obj_id, obl_obj_zones, moved_conds, pairs_dict = um
    mss = list(matching_subseqs(moved_conds, pairs_dict, obl_obj_id, obl_obj_zones, env))
    order_constrained_subseqs.extend(mss)

print("order_constrained_subseqs:")
print(order_constrained_subseqs)


for x in violating_sub_permutations(order_constrained_subseqs):
    print(set(x))



In [ ]:
s=time.time()
action_profile_with_norms=create_data(true_expression,env,name="demo",task1=task1,random_task=False,
                num_actionable=np.nan,num_repeat=250,verbose=False)
print ("Time Taken to complete job={:.2f}s\n".format(time.time()-s))

In [ ]:
data=[]
for itr,ap in action_profile_with_norms.items():
    for i in range(0,int(len(ap)/2)):
        data.append(tuple([ap[2*i],ap[2*i+1]]))
print ("Data Generated:")
for i in range(5):
    print(data[i])
    
with open('./demo/demo_data.sv', 'wb') as fp:
    pickle.dump(data, fp)

### 3. Run MCMC Algorithm to learn expressions from the data created earlier
1. rf is the relevance discounting factor for irrelevant expressions.
2. sim_t is the similarity threshold: cos(E1,E2) above which p_accept is penalised.
3. sim_pen is the penalty imposed if the above threshold is crossed.

In [ ]:
# Different parameters for MCMC sequence
rf=0.5 #To negate relevance logic use np.nan
sim_t=0.8
sim_pen=0.7 #To negate similarity logic use 1

In [ ]:
s=time.time()
print ("Generating sequence")
exp_seq,lik_list=algorithm_1(data,env,task1,true_expression,q_dict,rule_dict,
                                       filename="demo/demo_mcmc_report",
                                       sim_threshold=sim_t,similarity_penalty=sim_pen,
                                       relevance_factor=rf,max_iterations=50000,verbose=False)
print ("\nTime Taken to complete job={:.2f}s\n".format(time.time()-s))

In [ ]:
learned_expressions=Counter(map(to_tuple,exp_seq[int(len(exp_seq)/2)+1:]))#Discarding 1 half as warmup
print ("Number of unique Norms in sequence={}".format(len(learned_expressions)))

In [ ]:
# Write top norms to file
filename="demo_top_norms"
top=learned_expressions.most_common()
t=sum(learned_expressions.values())
exists = os.path.isfile('./demo/{}.txt'.format(filename))
if exists==True:
    os.remove('./demo/{}.txt'.format(filename))
original = sys.stdout
for i in range(len(top)):
    exp=top[i]
    if (i%10==0):
        print("Rank:{} Norm has relative frequency={:.3f}%".format(i+1,exp[1]*100/t))
    sys.stdout = open('./demo/{}.txt'.format(filename), 'a+')
    print("\n\n\n************Rank:{}, %-Frequency={:.3f}%**********".format(i+1,exp[1]*100/t))
    print_expression(exp[0])
    print("*************************************************")
    sys.stdout=original

In [ ]:
# Visualise the frequency of top plots
sns.set_style("darkgrid")
fig,ax=plt.subplots(figsize=(8,6),dpi=100)
fig.suptitle('Frequency of Norms in the generated sequence for relevance_factor={}'.format(rf))

t_l=sum(learned_expressions.values())
ax.plot([x*100/t for x in sorted(learned_expressions.values())],"o-",c=(250/255,93/255,130/255,0.7),markerfacecolor=(250/255,18/255,72/255,0.77))
ax.set_ylabel("%-Frequency in sample of size={}".format(t))
ax.set_xlabel("Descending Frequency Rank of Norms from a total of {} Norms".format(len(learned_expressions)))
ax.title.set_text("Weak Inequality check for E_0:\nlog_Likelihood(expression)>=log_Likelihood(No-Norm)")
obl_rank=[] #Ascending order Rank
for rank,x in enumerate(learned_expressions.most_common(),1):
    if x[0][1][0] =="Obl":
        obl_rank.append(rank)
for rank in obl_rank:
    ax.scatter(x=len(learned_expressions)-rank,
               y=sorted(learned_expressions.values())[len(learned_expressions)-rank]*100/t,
               c='green',s=151,marker='p',alpha=0.88,label='Obligatory,Rank={}'.format(rank))
ax.legend();

### 4. Calculate Preciscion and Recall for the Learned Norms

  * #####  Precision = $\frac{|\ true-data\ ∩\ predicted-data\ |}{|\ true-data\ |}$
  * #####  Recall = $\frac{|\ true-data\ ∩\ predicted-data\ |}{|\ predicted-data\ |}$
  * #####  F_beta = $\frac{(1+\beta^2)\  . \ (precision\ *\ recall)}{(\beta^2.precision\ +\ recall)}$
   where,
   *           True Data = All Possible Action Profiles that can be produced by true/trace expression
   *           Predicted Data = All Possible Action Profiles that can be produced by learned expression

In [ ]:
# Calculate precision and recall of top_n norms from learned expressions
pr_result=performance(task1,env,true_expression,learned_expressions,
                      folder_name="demo",file_name="top_norm",
                      top_n=np.nan,beta=1,verbose=False)
pr_result.head()

### 5. Test the convergence of MCMC Chain

<img src="mcmc_convergence/B_W_def.jpeg" alt="Drawing" style="width: 700px;float: center"/> 

In [ ]:
n=10000 #Length of sequence after discarding warm-up part and splitting in half
m=10 #Number of sequences after splitting in half

sequence_list=[]
for i in tnrange(1,int(m/2+1),desc="Loop for Individual Chains"):
    print ("\n:::::::::::::::::::: FOR SEQUENCE {} ::::::::::::::::::::".format(i))
    exp_seq,lik_list=algorithm_1(data,env,task1,true_expression,q_dict,rule_dict,
                                       filename="demo/convergence/report_for_chain_{}".format(i),
                                       sim_threshold=sim_t,similarity_penalty=sim_pen,
                                       relevance_factor=rf,max_iterations=4*n,verbose=False)
    sequence_list.append(exp_seq)

In [ ]:
convergence_result=calculate_R(prepare_sequences(sequence_list,warmup=True),50)
convergence_result